In [ ]:
import polars as pl
from math import sqrt
import queue
import random
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("konivat/tree-of-life")
dfs = []
for _filename_ in os.listdir(path):
    df = pl.read_csv(os.path.join(path, _filename_))
    dfs.append(df)

In [ ]:
#
# Implementation of the following paper:
#
#@inproceedings{10.1145/1124772.1124851,
#               author = {Wang, Weixin and Wang, Hui and Dai, Guozhong and Wang, Hongan},
#               title = {Visualization of large hierarchical data by circle packing},
#               year = {2006},
#               isbn = {1595933727},
#               publisher = {Association for Computing Machinery},
#               address = {New York, NY, USA},
#               url = {https://doi.org/10.1145/1124772.1124851},
#               doi = {10.1145/1124772.1124851},
#               abstract = {In this paper a novel approach is described for tree visualization using nested circles. The brother nodes at the same level are represented by externally tangent circles; the tree nodes at different levels are displayed by using 2D nested circles or 3D nested cylinders. A new layout algorithm for tree structure is described. It provides a good overview for large data sets. It is easy to see all the branches and leaves of the tree. The new method has been applied to the visualization of file systems.},
#               booktitle = {Proceedings of the SIGCHI Conference on Human Factors in Computing Systems},
#               pages = {517–520},
#               numpages = {4},
#               keywords = {tree visualization, nested circles, file system, circle packing},
#               location = {Montr\'{e}al, Qu\'{e}bec, Canada},
#               series = {CHI '06} }
#
def packCircles(self, circles, epsilon=0.01):
    if   len(circles) == 0: return [], {}, {}
    elif len(circles) == 1: return [(0.0, 0.0, circles[0][2])], {}, {}
    elif len(circles) == 2: return [(0.0, 0.0, circles[0][2]), (circles[0][2] + circles[1][2], 0.0, circles[1][2])], {}, {}
    else:
        placed = []
        placed.append((0.0, 0.0, circles[0][2]))
        placed.append((circles[0][2] + circles[1][2], 0.0, circles[1][2]))
        c0, c1, r = placed[0], placed[1], circles[2][2]
        xy0, xy1 = self.overlappingCirclesIntersections((c0[0],c0[1],c0[2]+r),(c1[0],c1[1],c1[2]+r))
        placed.append((xy0[0], xy0[1], r))
        # Create the forward and backward chains
        fwd,    bck    = {}, {}
        fwd[0], bck[0] = 1, 2
        fwd[1], bck[1] = 2, 0
        fwd[2], bck[2] = 0, 1
        # List all of the circles in the chain
        in_chain = set([0, 1, 2])
        # Create a priority queue -- this will return the closest point to the origin
        nearest  = queue.PriorityQueue()
        nearest.put((0.0, 0))
        nearest.put((sqrt(placed[1][0]**2 + placed[1][1]**2), 1))
        nearest.put((sqrt(placed[2][0]**2 + placed[2][1]**2), 2))
        for i in range(3, len(circles)):
            c = circles[i]
            while nearest.queue[0][1] not in in_chain: nearest.get() # find the next nearest circle that is still in the chain
            c0_i, c1_i = nearest.queue[0][1], fwd[nearest.queue[0][1]]
            c0,   c1   = placed[c0_i], placed[c1_i]
            xy0,  xy1  = self.overlappingCirclesIntersections((c0[0],c0[1],c0[2]+c[2]),(c1[0],c1[1],c1[2]+c[2]))
            c_possible = (xy0[0], xy0[1], c[2])
            c_overlaps = None
            nxt_i      = fwd[c1_i]
            while nxt_i != c0_i:
                if self.circlesOverlap((c_possible[0], c_possible[1], c_possible[2]-epsilon), placed[nxt_i]):
                    print(f'c0, c1 = {c_possible}, {placed[nxt_i]} # {i} {c_overlaps}')
                    c_overlaps = placed[nxt_i]
                else: print('.', end='')
                nxt_i = fwd[nxt_i]
            if c_overlaps is not None:
                placed.append((xy1[0], xy1[1], c[2]))
                fwd[i]    = c1_i
                fwd[c0_i] = i
                bck[i]    = c0_i
                bck[c1_i] = i
                in_chain.add(i)
                nearest.put((sqrt(c_possible[0]**2 + c_possible[1]**2), i))
        return placed, fwd, bck

_circles_ = []
for i in range(7): _circles_.append((0.0, 0.0, 1.0 + 3*random.random()))
_packed_, _fwd_, _bck_ = packCircles(rt, _circles_)

xs, ys, ws, hs = -20.0, -20.0, 40.0, 40.0
def svgCircles(_chn_, _color_='#000000'):
    svg = [f'<svg x="0" y="0" width="500" height="500" viewBox="{xs} {ys} {ws} {hs}" xmlns="http://www.w3.org/2000/svg">']
    svg.append(f'<rect x="{xs}" y="{ys}" width="{ws}" height="{hs}" fill="#ffffff" />')
    svg.append(f'<line x1="-1" y1="-1" x2="1" y2="1" stroke="black" stroke-width="0.2" />')
    svg.append(f'<line x1="1" y1="-1" x2="-1" y2="1" stroke="black" stroke-width="0.2" />')
    for i in range(len(_packed_)):
        _c_ = _packed_[i]
        svg.append(f'<circle cx="{_c_[0]}" cy="{_c_[1]}" r="{_c_[2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.2" />')
    _index_ = list(_chn_.keys())[0]
    for i in range(len(_chn_.keys())+1):
        _index_next_ = _chn_[_index_]
        xy0, xy1 = _packed_[_index_], _packed_[_index_next_]
        svg.append(f'<line x1="{xy0[0]}" y1="{xy0[1]}" x2="{xy1[0]}" y2="{xy1[1]}" stroke="{_color_}" stroke-width="0.2" />')
        uv       = rt.unitVector((xy0, xy1))
        perp     = (-uv[1], uv[0])
        svg.append(f'<line x1="{xy1[0]}" y1="{xy1[1]}" x2="{xy1[0] - 1*uv[0] + 0.5*perp[0]}" y2="{xy1[1] - 1*uv[1] + 0.5*perp[1]}" stroke="{_color_}" stroke-width="0.1" />')
        svg.append(f'<line x1="{xy1[0]}" y1="{xy1[1]}" x2="{xy1[0] - 1*uv[0] - 0.5*perp[0]}" y2="{xy1[1] - 1*uv[1] - 0.5*perp[1]}" stroke="{_color_}" stroke-width="0.1" />')
        _index_  = _index_next_
    svg.append('</svg>')
    return ''.join(svg)
rt.tile([svgCircles(_fwd_), svgCircles(_bck_, '#ff0000')], spacer=10)

In [ ]:
c0, c1 = (4.22495469514033, -0.5658098410611183, 2.023711641099232), (5.405109143156395, 0.0, 3.166147567572237) # 4
_svg_ = [f'<svg x="0" y="0" width="500" height="500" viewBox="{xs} {ys} {ws} {hs}" xmlns="http://www.w3.org/2000/svg">',]
_svg_.append(f'<rect x="{xs}" y="{ys}" width="{ws}" height="{hs}" fill="#ffffff" />')
_svg_.append(f'<line x1="-1" y1="-1" x2="1" y2="1" stroke="black" stroke-width="0.2" />')
_svg_.append(f'<line x1="1" y1="-1" x2="-1" y2="1" stroke="black" stroke-width="0.2" />')
_svg_.append(f'<circle cx="{c0[0]}" cy="{c0[1]}" r="{c0[2]}" fill="none" stroke="black" stroke-width="0.2" />')
_svg_.append(f'<circle cx="{c1[0]}" cy="{c1[1]}" r="{c1[2]}" fill="none" stroke="black" stroke-width="0.2" />')
_svg_.append('</svg>')
rt.tile([''.join(_svg_)])